In [1]:
# import pandas as pd
import databricks.koalas as kpd
import sklearn as sk
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.offline import plot
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("customer_segmentation") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#inititate Plotly
pyoff.init_notebook_mode()
# pd.options.mode.chained_assignment = None

df = kpd.read_csv("/FileStore/tables/master_table.csv")

print(df.dtypes)
print(df.head(5))

_c0 int32
Invoice_Number int32
Order_Number int32
Order_Date_Id datetime64[ns]
SKU_Id int32
Plant_Id int32
Customer_Id int32
Sourced_Product object
Sales_Price float64
Delivery_Charge float64
Extra_Charges float64
Standard_Cost float64
Quantity int32
Ship_Via object
Sales_Id int32
Invoice_Year int32
Invoice_Month int32
Customer_SoldTo_State object
Customer_SoldTo_Zip object
Business_Unit_Code int32
Business_Unit_Name object
Plant_State object
Plant_1Digit_Code object
dtype: object
 _c0 Invoice_Number Order_Number Order_Date_Id SKU_Id Plant_Id Customer_Id Sourced_Product Sales_Price Delivery_Charge Extra_Charges Standard_Cost Quantity Ship_Via Sales_Id Invoice_Year Invoice_Month Customer_SoldTo_State Customer_SoldTo_Zip Business_Unit_Code Business_Unit_Name Plant_State Plant_1Digit_Code
0 0 1 9640 2019-02-07 306 11 738 Y 1217.0 84.10 0.00 805.33 1 OT 1 2019 3 IA 52001-7004 11 Home Furnishings KS K
1 1 1 80132 2018-09-04 658 8 895 N 10100.0 635.00 0.00 7070.30 10 OT 2 2018 10 GA 30082 11 Home Furnishings TX E
2 2 2 85541 2017-12-28 540 11 833 Y 495.0 39.75 7.55 326.75 1 OT 4 2018 2 IA 52001 11 Home Furnishings KS K
3 3 2 86229 2018-09-10 209 8 503 N 760.0 73.80 0.00 570.06 2 OT 5 2018 10 IA 52001 11 Home Furnishings TX E
4 4 3 14610 2018-01-24 713 11 717 Y 105.0 6.00 1.14 84.75 1 OT 6 2018 2 IA 52001 11 Home Furnishings KS K

In [2]:
# Remove index column and change order date to datetime
df = df.drop(columns=['_c0'])

df['Order_Date_Id'] = kpd.to_datetime(df['Order_Date_Id'])
print(df.dtypes)
print(df.shape)

##### RFM Feature engineering #####

# Get only rows with a valid sales price value
df = df.query('Sales_Price != 0')
print(df.shape)

# Drop dupes
df = df.drop_duplicates()
print(df.shape)

Invoice_Number int32
Order_Number int32
Order_Date_Id datetime64[ns]
SKU_Id int32
Plant_Id int32
Customer_Id int32
Sourced_Product object
Sales_Price float64
Delivery_Charge float64
Extra_Charges float64
Standard_Cost float64
Quantity int32
Ship_Via object
Sales_Id int32
Invoice_Year int32
Invoice_Month int32
Customer_SoldTo_State object
Customer_SoldTo_Zip object
Business_Unit_Code int32
Business_Unit_Name object
Plant_State object
Plant_1Digit_Code object
dtype: object
(1000, 22)
(792, 22)
(792, 22)

In [3]:
### Recency ###

# Create df with unique customer IDs
df_cust = kpd.DataFrame(df['Customer_Id'].unique())
df_cust.columns = ['Customer_Id']
print(df_cust)

Customer_Id
0 833
1 148
2 471
3 496
4 463
5 540
6 243
7 897
8 737
9 516
10 85
11 251
12 137
13 451
14 879
15 65
16 883
17 255
18 481
19 970
20 898
21 799
22 296
23 513
24 918
25 673
26 362
27 974
28 613
29 321
30 950
31 155
32 530
33 368
34 101
35 772
36 756
37 874
38 81
39 830
40 939
41 28
42 210
43 762
44 497
45 412
46 787
47 842
48 988
49 731
50 76
51 688
52 723
53 26
54 332
55 831
56 916
57 271
58 192
59 844
60 253
61 460
62 12
63 223
64 860
65 738
66 548
67 602
68 91
69 578
70 926
71 663
72 333
73 822
74 707
75 875
76 209
77 727
78 685
79 330
80 22
81 686
82 230
83 319
84 122
85 493
86 758
87 232
88 225
89 360
90 346
91 367
92 246
93 855
94 599
95 519
96 654
97 224
98 111
99 416
100 827
101 177
102 556
103 711
104 132
105 671
106 665
107 185
108 305
109 618
110 696
111 882
112 906
113 386
114 603
115 912
116 774
117 704
118 512
119 1
120 896
121 206
122 297
123 777
124 52
125 992
126 674
127 212
128 274
129 182
130 689
131 699
132 13
133 348
134 280
135 86
136 760
137 6
138 16
139 205
140 572
141 168
142 852
143 3
144 925
145 998
146 972
147 328
148 867
149 308
150 164
151 470
152 40
153 839
154 429
155 500
156 169
157 479
158 283
159 431
160 340
161 295
162 250
163 582
164 377
165 432
166 920
167 868
168 647
169 966
170 676
171 944
172 923
173 292
174 120
175 559
176 969
177 491
178 776
179 846
180 545
181 235
182 732
183 452
184 266
185 5
186 959
187 729
188 430
189 989
190 560
191 553
192 542
193 419
194 780
195 644
196 257
197 227
198 117
199 886
200 598
201 425
202 931
203 484
204 447
205 845
206 15
207 929
208 511
209 262
210 112
211 849
212 327
213 775
214 366
215 358
216 854
217 189
218 207
219 862
220 590
221 383
222 37
223 127
224 523
225 937
226 502
227 486
228 197
229 531
230 462
231 619
232 202
233 17
234 9
235 609
236 398
237 505
238 538
239 175
240 387
241 381
242 498
243 850
244 721
245 594
246 880
247 503
248 610
249 614
250 761
251 739
252 568
253 900
254 485
255 870
256 829
257 456
258 884
259 733
260 948
261 986
262 326
263 229
264 301
265 713
266 278
267 114
268 173
269 100
270 450
271 241
272 717
273 495
274 987
275 39
276 922
277 719
278 679
279 520
280 672
281 380
282 162
283 549
284 877
285 648
286 238
287 244
288 624
289 446
290 84
291 856
292 664
293 314
294 691
295 627
296 728
297 87
298 565
299 649
300 273
301 585
302 439
303 443
304 51
305 994
306 465
307 194
308 794
309 438
310 129
311 467
312 448
313 400
314 661
315 533
316 524
317 693
318 239
319 264
320 234
321 312
322 635
323 885
324 982
325 395
326 403
327 361
328 10
329 848
330 814
331 658
332 771
333 951
334 404
335 267
336 527
337 977
338 793
339 569
340 45
341 938
342 769
343 812
344 759
345 952
346 946
347 38
348 364
349 249
350 181
351 82
352 576
353 945
354 167
355 394
356 80
357 240
358 889
359 25
360 73
361 763
362 427
363 113
364 895
365 645
366 893
367 160
368 818
369 408
370 70
371 921
372 125
373 62
374 351
375 480
376 697
377 95
378 789
379 390
380 741
381 558
382 464
383 323
384 550
385 866
386 352
387 335
388 492
389 941
390 98
391 911
392 195
393 714
394 678
395 490
396 595
397 345
398 287
399 767
400 790
401 990
402 863
403 203
404 221
405 662
406 835
407 640
408 859
409 341
410 414
411 712
412 433
413 801
414 751
415 515
416 488
417 735
418 376
419 200
420 272
421 940
422 254
423 639
424 56
425 487
426 378
427 109
428 534
429 213
430 228
431 997
432 655
433 478
434 574
435 426
436 749
437 354
438 708
439 170
440 783
441 611
442 370
443 913
444 837
445 888
446 33
447 11
448 872
449 110
450 150
451 825
452 508
453 106
454 68
455 71
456 567
457 525
458 561
459 805
460 14
461 198
462 546
463 349
464 344
465 975
466 832
467 373
468 123
469 379
470 342
471 199
472 690
473 270
474 902
475 499
476 337
477 566
478 466
479 506
480 119
481 892
482 778
483 765
484 79
485 149
486 999
487 365
488 773
489 294
490 573
491 571
492 124
493 99
494 985
495 184
496 302
497 815
498 557
499 700
500 66
501 682
502 46
503 67
504 802
505 605
506 275
507 873
508 535
509 311
510 628
511 964
512 174
513 996
514 864
515 716
516 562
517 399
518 252

In [4]:
# Get the most recent date for each customer in seperate df
df_mp = df.groupby('Customer_Id').Order_Date_Id.max().reset_index()
df_mp.columns = ['Customer_Id','MostRecent']
print(df_mp.dtypes)

Customer_Id int32
MostRecent datetime64[ns]
dtype: object

In [5]:
# Take most recent date in dataset to find recency for each observation
df_mp['MostRecentMax'] = df_mp.MostRecent.max()
print(df_mp['MostRecentMax'])

df_mp['Recency'] = (df_mp['MostRecentMax'] - df_mp['MostRecent'])/86400
df_mp['Recency'] = df_mp['Recency'].astype('int64')
print(df_mp)


# Merge dataframes
df_cust = kpd.merge(df_cust, df_mp[['Customer_Id','Recency']], on='Customer_Id')

df_cust.head()

,Customer_Id,Recency
0,833,7
1,148,304
2,471,516
3,496,7
4,463,236


In [6]:
# Recency histogram
print(df_cust['Recency'].plot.hist())

In [7]:
# Find optimal clusters for Recency score, using elbow method
# from sklearn.cluster import KMeans
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

# Convert Koala dataframe to Spark dataframe
df_rec = df_cust.to_spark()

# Prepare for KMeans algo
rec_feat = ['Recency']
vecAssembler = VectorAssembler(inputCols=rec_feat, outputCol="features")
df_kmeansR = vecAssembler.transform(df_rec).select('Customer_Id','features')
cost = np.zeros(10)

In [8]:
# Fit and plot Elbow curve
for k in range(2,10):
    kmeansR = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    modelR = kmeansR.fit(df_kmeansR.sample(False,0.1, seed=42))
    cost[k] = modelR.computeCost(df_kmeansR)
    
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),cost[2:10])
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [9]:
# Build 3 clusters (elbow method) for Recency and add it to df
kpd.set_option('compute.ops_on_diff_frames', True)

# Predict clusters in spark dataframe, convert to koalas, and grab predictions
kmeansR = KMeans(k=3)
modelR = kmeansR.fit(df_kmeansR)
k_means_recency = modelR.transform(df_kmeansR).select('Customer_Id', 'prediction')
rows = k_means_recency.collect()
df_recency_pred = spark.createDataFrame(data=rows)
df_recency_pred = kpd.DataFrame(df_recency_pred)
df_cust['RecencyCluster'] = df_recency_pred['prediction']

kpd.reset_option('compute.ops_on_diff_frames')

In [10]:
# Function for ordering cluster numbers (0 cluster number is low recency, 2 is high recency)
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    #new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by = target_field_name ,ascending = ascending).reset_index(drop=True) 
    index = df_new.index.to_numpy()
    df_index = kpd.DataFrame(data = index, columns = ['index'])
    df_new = kpd.concat([df_new, df_index], axis = 1)
    # Take index of predictions 
    df_final = kpd.merge(df, df_new[[cluster_field_name,'index']], on = cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis=1)
    df_final = df_final.rename(columns = {"index":cluster_field_name})
    return df_final

df_cust = order_cluster('RecencyCluster', 'Recency', df_cust, False)
df_cust.describe()

,Customer_Id,Recency,RecencyCluster
count,542.000000,542.000000,542.000000
mean,508.258303,155.725092,1.555351
std,287.916680,138.018390,0.657446
min,1.000000,0.000000,0.000000
25%,255.000000,76.000000,1.000000
50%,505.000000,96.000000,2.000000
75%,762.000000,238.000000,2.000000
max,999.000000,567.000000,2.000000


In [11]:
### Frequency ###

# Order counts per customer
df_freq = df.groupby('Customer_Id').Order_Date_Id.count().reset_index()
df_freq.columns = ['Customer_Id','Frequency']

# Add to main df
df_cust = kpd.merge(df_cust, df_freq, on='Customer_Id')
print(df_cust)

Customer_Id Recency RecencyCluster Frequency
0 833 7 2 2
1 496 7 2 2
2 540 4 2 1
3 243 96 2 1
4 897 46 2 1
5 251 118 2 3
6 451 140 2 2
7 879 34 2 2
8 65 81 2 1
9 883 122 2 1
10 255 6 2 1
11 970 77 2 1
12 898 10 2 1
13 799 96 2 1
14 296 80 2 3
15 918 111 2 1
16 613 41 2 1
17 321 34 2 2
18 530 4 2 2
19 368 89 2 1
20 101 14 2 3
21 772 0 2 1
22 756 124 2 1
23 874 34 2 2
24 939 118 2 2
25 28 84 2 1
26 210 111 2 1
27 412 89 2 2
28 787 21 2 2
29 842 45 2 1
30 731 103 2 3
31 76 26 2 1
32 688 21 2 1
33 723 82 2 2
34 26 82 2 1
35 332 111 2 1
36 831 4 2 2
37 916 40 2 2
38 271 124 2 2
39 253 82 2 1
40 12 83 2 3
41 860 34 2 1
42 738 88 2 1
43 548 83 2 1
44 602 118 2 1
45 91 34 2 4
46 926 34 2 1
47 663 89 2 1
48 822 34 2 2
49 875 77 2 3
50 685 91 2 1
51 686 87 2 1
52 230 116 2 1
53 319 34 2 3
54 122 77 2 2
55 232 82 2 1
56 225 90 2 1
57 367 118 2 1
58 246 81 2 1
59 654 39 2 1
60 224 118 2 1
61 111 77 2 1
62 416 82 2 1
63 177 84 2 2
64 711 108 2 1
65 132 119 2 1
66 671 77 2 1
67 665 34 2 2
68 185 108 2 3
69 305 82 2 1
70 618 7 2 1
71 882 82 2 1
72 906 34 2 2
73 603 46 2 1
74 774 82 2 1
75 704 77 2 1
76 512 97 2 1
77 1 77 2 2
78 896 34 2 3
79 297 89 2 5
80 777 41 2 2
81 52 40 2 2
82 992 82 2 1
83 674 102 2 2
84 212 111 2 1
85 274 83 2 3
86 689 47 2 2
87 699 96 2 2
88 13 76 2 1
89 280 84 2 1
90 86 80 2 3
91 760 74 2 1
92 6 4 2 1
93 16 3 2 1
94 205 108 2 1
95 572 89 2 2
96 168 46 2 2
97 3 82 2 1
98 998 84 2 1
99 972 47 2 1
100 328 4 2 4
101 867 108 2 1
102 308 89 2 1
103 164 0 2 2
104 470 83 2 3
105 839 147 2 1
106 479 96 2 1
107 283 83 2 1
108 340 82 2 1
109 295 126 2 2
110 250 82 2 2
111 582 96 2 2
112 432 0 2 2
113 920 34 2 2
114 647 84 2 2
115 966 136 2 1
116 944 26 2 3
117 923 96 2 1
118 292 80 2 1
119 120 108 2 1
120 559 84 2 1
121 491 110 2 1
122 846 46 2 2
123 235 76 2 2
124 452 84 2 1
125 959 34 2 4
126 729 46 2 2
127 989 118 2 2
128 560 18 2 2
129 553 14 2 1
130 419 77 2 2
131 780 46 2 1
132 644 0 2 1
133 227 46 2 1
134 117 116 2 1
135 484 74 2 3
136 447 13 2 1
137 845 118 2 1
138 15 84 2 1
139 262 82 2 1
140 112 91 2 1
141 849 84 2 1
142 327 34 2 3
143 775 116 2 3
144 366 82 2 2
145 189 74 2 1
146 207 82 2 2
147 862 7 2 2
148 590 4 2 2
149 37 82 2 1
150 127 82 2 1
151 486 147 2 1
152 197 82 2 2
153 531 83 2 2
154 462 131 2 2
155 202 21 2 1
156 17 82 2 1
157 609 108 2 2
158 505 0 2 1
159 538 81 2 1
160 175 42 2 3
161 381 46 2 2
162 498 25 2 2
163 850 34 2 2
164 610 46 2 2
165 614 80 2 1
166 761 4 2 2
167 739 118 2 1
168 568 96 2 3
169 900 109 2 2
170 870 81 2 2
171 829 80 2 3
172 456 82 2 1
173 884 83 2 2
174 948 88 2 2
175 986 80 2 2
176 326 103 2 3
177 229 96 2 1
178 450 81 2 1
179 495 118 2 1
180 987 83 2 1
181 719 96 2 2
182 679 84 2 1
183 520 34 2 2
184 162 31 2 1
185 549 83 2 1
186 877 77 2 2
187 238 87 2 2
188 244 34 2 2
189 446 80 2 1
190 84 82 2 1
191 664 34 2 1
192 314 80 2 1
193 691 140 2 1
194 627 83 2 1
195 87 48 2 2
196 273 46 2 2
197 585 82 2 1
198 439 110 2 1
199 51 84 2 2
200 465 75 2 3
201 794 84 2 1
202 438 136 2 1
203 129 74 2 1
204 448 18 2 1
205 400 6 2 1
206 693 124 2 1
207 239 84 2 2
208 264 140 2 1
209 312 25 2 2
210 635 80 2 1
211 982 32 2 1
212 361 0 2 2
213 10 83 2 2
214 848 84 2 1
215 658 82 2 1
216 771 13 2 2
217 404 112 2 1
218 267 34 2 2
219 527 90 2 2
220 793 3 2 3
221 45 80 2 2
222 938 77 2 2
223 812 82 2 2
224 952 84 2 1
225 946 89 2 1
226 38 74 2 3
227 364 124 2 1
228 249 46 2 2
229 181 96 2 1
230 82 82 2 1
231 945 84 2 1
232 167 77 2 1
233 394 126 2 1
234 80 42 2 3
235 889 140 2 2
236 25 145 2 2
237 73 151 2 2
238 763 5 2 1
239 113 77 2 1
240 645 42 2 2
241 893 4 2 1
242 160 140 2 2
243 70 115 2 1
244 921 0 2 1
245 351 110 2 1
246 480 111 2 1
247 697 82 2 3
248 789 3 2 2
249 390 7 2 2
250 741 81 2 1
251 323 75 2 1
252 866 80 2 2
253 352 84 2 1
254 941 74 2 1
255 911 4 2 2
256 195 84 2 1
257 714 83 2 1
258 490 40 2 2
259 595 115 2 2
260 287 4 2 1
261 767 82 2 5
262 221 124 2 1
263 662 81 2 1
264 859 47 2 2
265 341 108 2 1
266 712 34 2 2
267 433 140 2 1
268 751 122 2 1
269 515 136 2 1
270 488 116 2 3

In [12]:
# Frequency histogram
print(df_cust['Frequency'].plot.hist())

In [13]:
# Find optimal clusters for Frequency score, using elbow method

# Convert Koala dataframe to Spark dataframe
df_freq = df_cust.to_spark()

freq_feat = ['Frequency']
vecAssemblerF = VectorAssembler(inputCols=freq_feat, outputCol="features")
df_kmeansF = vecAssemblerF.transform(df_freq).select('Customer_Id','features')
cost = np.zeros(10)

In [14]:
# Fit and plot Elbow curve
for k in range(2,10):
    kmeansF = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    modelF = kmeansF.fit(df_kmeansF.sample(False,0.1, seed=42))
    cost[k] = modelF.computeCost(df_kmeansF)
    
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),cost[2:10])
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [15]:
# Build 3 clusters (elbow method) for Frequency and add it to df
kpd.set_option('compute.ops_on_diff_frames', True)

# Predict clusters in spark dataframe, convert to koalas, and grab predictions
kmeans = KMeans(k=3)
modelF = kmeans.fit(df_kmeansF)
k_means_frequency = modelF.transform(df_kmeansF).select('Customer_Id', 'prediction')
rows = k_means_frequency.collect()
df_frequency_pred = spark.createDataFrame(rows)
df_frequency_pred = kpd.DataFrame(df_frequency_pred)
df_cust['FrequencyCluster'] = df_frequency_pred['prediction']

kpd.reset_option('compute.ops_on_diff_frames')

# Order Frequency clusters
df_cust = order_cluster('FrequencyCluster', 'Frequency', df_cust, True)
print(df_cust)

# Summary about Frequency Clusters
df_cust.describe()

,Customer_Id,Recency,RecencyCluster,Frequency,FrequencyCluster
count,542.000000,542.000000,542.000000,542.000000,542.000000
mean,508.258303,155.725092,1.555351,1.461255,0.444649
std,287.916680,138.018390,0.657446,0.699467,0.643235
min,1.000000,0.000000,0.000000,1.000000,0.000000
25%,255.000000,76.000000,1.000000,1.000000,0.000000
50%,505.000000,96.000000,2.000000,1.000000,0.000000
75%,762.000000,238.000000,2.000000,2.000000,1.000000
max,999.000000,567.000000,2.000000,5.000000,2.000000


In [16]:
### Revenue ###

# Find revenue for each customer
df['Revenue'] = df['Sales_Price'] * df['Quantity']
df_rev = df.groupby('Customer_Id').Revenue.sum().reset_index()

# Add to main df
df_cust = kpd.merge(df_cust, df_rev, on='Customer_Id')
print(df_cust)

Customer_Id Recency RecencyCluster Frequency FrequencyCluster Revenue
0 833 7 2 2 1 1442.00
1 496 7 2 2 1 2585.00
2 451 140 2 2 1 2754.00
3 879 34 2 2 1 1262.00
4 321 34 2 2 1 741.00
5 530 4 2 2 1 1272.00
6 874 34 2 2 1 815.00
7 939 118 2 2 1 958.00
8 412 89 2 2 1 2670.00
9 787 21 2 2 1 2239.00
10 723 82 2 2 1 1671.00
11 831 4 2 2 1 28653.00
12 916 40 2 2 1 669.00
13 271 124 2 2 1 746.00
14 822 34 2 2 1 628.00
15 122 77 2 2 1 15173.00
16 177 84 2 2 1 1230.00
17 665 34 2 2 1 463.00
18 906 34 2 2 1 710.00
19 1 77 2 2 1 3909.00
20 777 41 2 2 1 1257.95
21 52 40 2 2 1 2506.00
22 674 102 2 2 1 625.00
23 689 47 2 2 1 1378.00
24 699 96 2 2 1 5400.00
25 572 89 2 2 1 2560.00
26 168 46 2 2 1 799.50
27 164 0 2 2 1 1332.00
28 295 126 2 2 1 5117.00
29 250 82 2 2 1 1021.00
30 582 96 2 2 1 1145.50
31 432 0 2 2 1 1038.00
32 920 34 2 2 1 1753.00
33 647 84 2 2 1 1372.00
34 846 46 2 2 1 1559.60
35 235 76 2 2 1 994.00
36 729 46 2 2 1 783.00
37 989 118 2 2 1 3047.00
38 560 18 2 2 1 2465.00
39 419 77 2 2 1 1173.00
40 366 82 2 2 1 1323.20
41 207 82 2 2 1 643.00
42 862 7 2 2 1 414.00
43 590 4 2 2 1 1728.00
44 197 82 2 2 1 1290.00
45 531 83 2 2 1 2074.00
46 462 131 2 2 1 1556.00
47 609 108 2 2 1 7664.00
48 381 46 2 2 1 1050.00
49 498 25 2 2 1 1193.00
50 850 34 2 2 1 964.00
51 610 46 2 2 1 948.00
52 761 4 2 2 1 1811.00
53 900 109 2 2 1 1592.00
54 870 81 2 2 1 2279.00
55 884 83 2 2 1 2125.00
56 948 88 2 2 1 1411.00
57 986 80 2 2 1 2005.00
58 719 96 2 2 1 2605.80
59 520 34 2 2 1 2624.00
60 877 77 2 2 1 214.00
61 238 87 2 2 1 705.45
62 244 34 2 2 1 155.00
63 87 48 2 2 1 2638.00
64 273 46 2 2 1 849.00
65 51 84 2 2 1 1480.00
66 239 84 2 2 1 839.00
67 312 25 2 2 1 1020.00
68 361 0 2 2 1 2046.00
69 10 83 2 2 1 5783.00
70 771 13 2 2 1 1708.00
71 267 34 2 2 1 1817.00
72 527 90 2 2 1 1079.00
73 45 80 2 2 1 4566.00
74 938 77 2 2 1 618.00
75 812 82 2 2 1 1185.60
76 249 46 2 2 1 251.00
77 889 140 2 2 1 2105.00
78 25 145 2 2 1 1545.00
79 73 151 2 2 1 2478.00
80 645 42 2 2 1 1515.10
81 160 140 2 2 1 845.00
82 789 3 2 2 1 545.00
83 390 7 2 2 1 1725.00
84 866 80 2 2 1 2572.00
85 911 4 2 2 1 1730.00
86 490 40 2 2 1 734.00
87 595 115 2 2 1 12321.00
88 859 47 2 2 1 1845.00
89 712 34 2 2 1 863.00
90 735 140 2 2 1 1520.00
91 376 6 2 2 1 1593.00
92 940 82 2 2 1 14190.00
93 655 4 2 2 1 229.00
94 426 89 2 2 1 7200.00
95 749 112 2 2 1 410.00
96 708 118 2 2 1 865.80
97 783 67 2 2 1 2161.00
98 370 21 2 2 1 1374.00
99 110 0 2 2 1 1281.00
100 508 129 2 2 1 920.00
101 805 84 2 2 1 1155.00
102 349 96 2 2 1 1138.00
103 832 91 2 2 1 700.00
104 373 32 2 2 1 1015.00
105 379 0 2 2 1 4436.00
106 690 38 2 2 1 1583.00
107 499 4 2 2 1 634.00
108 466 80 2 2 1 994.00
109 892 82 2 2 1 2253.40
110 79 76 2 2 1 1436.00
111 124 40 2 2 1 1196.00
112 682 41 2 2 1 2752.00
113 46 19 2 2 1 2526.20
114 605 84 2 2 1 653.00
115 964 89 2 2 1 1814.00
116 174 84 2 2 1 894.00
117 716 7 2 2 1 423.00
118 878 7 2 2 1 2080.00
119 134 74 2 2 1 1033.00
120 521 74 2 2 1 1191.00
121 776 467 0 2 1 607.00
122 240 494 0 2 1 800.00
123 988 229 1 2 1 348.50
124 460 238 1 2 1 1885.00
125 223 237 1 2 1 3140.00
126 209 236 1 2 1 580.00
127 758 237 1 2 1 545.00
128 360 238 1 2 1 1340.00
129 855 238 1 2 1 1395.00
130 696 238 1 2 1 1385.00
131 182 229 1 2 1 442.00
132 169 241 1 2 1 685.00
133 676 248 1 2 1 1355.00
134 732 237 1 2 1 75433.00
135 511 230 1 2 1 630.00
136 383 237 1 2 1 1030.00
137 594 230 1 2 1 960.00
138 713 244 1 2 1 31280.00
139 467 238 1 2 1 985.00
140 533 241 1 2 1 1239.00
141 234 270 1 2 1 770.00
142 125 238 1 2 1 2315.00
143 990 238 1 2 1 974.00
144 801 237 1 2 1 2310.00
145 611 249 1 2 1 1013.25
146 99 236 1 2 1 778.00
147 700 238 1 2 1 5640.75
148 399 357 1 2 1 900.00
149 755 255 1 2 1 1226.00
150 643 237 1 2 1 22680.00
151 540 4 2 1 0 1165.00
152 243 96 2 1 0 596.00
153 897 46 2 1 0 387.00
154 65 81 2 1 0 751.00
155 883 122 2 1 0 790.00
156 255 6 2 1 0 854.00
157 970 77 2 1 0 218.00
158 898 10 2 1 0 1988.00
159 799 96 2 1 0 1308.00
160 918 111 2 1 0 618.00
161 613 41 2 1 0 327.00
162 368 89 2 1 0 932.00
163

In [17]:
# Monetary Value histogram
print(df_cust['Revenue'].plot.hist())

In [18]:
# Find optimal clusters for Revenue score, using elbow method

# Convert Koala dataframe to Spark dataframe
df_rev = df_cust.to_spark()

rev_feat = ['Revenue']
vecAssemblerM = VectorAssembler(inputCols=rev_feat, outputCol="features")
df_kmeansM = vecAssemblerM.transform(df_rev).select('Customer_Id','features')
cost = np.zeros(10)

In [19]:
# Fit and plot Elbow curve
for k in range(2,10):
    kmeansM = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    modelM = kmeansM.fit(df_kmeansM.sample(False,0.1, seed=42))
    cost[k] = modelM.computeCost(df_kmeansM)
    
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),cost[2:10])
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [20]:
# Build 3 clusters (elbow method) for Revenue and add it to df
kpd.set_option('compute.ops_on_diff_frames', True)

# Predict clusters in spark dataframe, convert to koalas, and grab predictions
kmeansM = KMeans(k=3)
modelM = kmeansM.fit(df_kmeansM)
k_means_revenue = modelM.transform(df_kmeansM).select('Customer_Id', 'prediction')
rows = k_means_revenue.collect()
df_revenue_pred = spark.createDataFrame(rows)
df_revenue_pred = kpd.DataFrame(df_revenue_pred)
df_cust['RevenueCluster'] = df_revenue_pred['prediction']

kpd.reset_option('compute.ops_on_diff_frames')

# Order Revenue clusters
df_cust = order_cluster('RevenueCluster', 'Revenue', df_cust, True)
print(df_cust)

# Summary about Revenue Clusters
df_cust.describe()

,Customer_Id,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster
count,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000
mean,508.258303,155.725092,1.555351,1.461255,0.444649,2931.773801,0.012915
std,287.916680,138.018390,0.657446,0.699467,0.643235,12481.113026,0.142006
min,1.000000,0.000000,0.000000,1.000000,0.000000,80.000000,0.000000
25%,255.000000,76.000000,1.000000,1.000000,0.000000,460.000000,0.000000
50%,505.000000,96.000000,2.000000,1.000000,0.000000,849.000000,0.000000
75%,762.000000,238.000000,2.000000,2.000000,1.000000,1733.000000,0.000000
max,999.000000,567.000000,2.000000,5.000000,2.000000,175281.000000,2.000000


In [21]:
# Get Overall Score and look at mean
df_cust['OverallScore'] = df_cust['RecencyCluster'] + df_cust['FrequencyCluster'] + df_cust['RevenueCluster']
RFM = ['Recency','Frequency','Revenue']
df_cust.groupby('OverallScore')[RFM].mean()

,Recency,Frequency,Revenue
OverallScore,,,
0,500.765957,1.000000,4325.061702
6,75.000000,3.000000,175281.000000
5,82.000000,5.000000,115592.200000
1,251.454545,1.018182,1039.069364
3,68.579365,2.015873,3862.032540
2,101.068807,1.133028,1681.510550
4,60.923077,3.179487,3266.340256


In [22]:
# Name buckets
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

df_cust = df_cust.to_spark()

def func(OverallScore):
    if OverallScore < 2:
        return 'Low-Value'
    if OverallScore >= 2 and OverallScore < 4:
        return 'Mid-Value'
    if OverallScore >= 4:  
        return 'High-Value'

func_udf = udf(func, StringType())
df_cust = df_cust.withColumn('Segment',func_udf(df_cust['OverallScore']))

df_cust = kpd.DataFrame(df_cust)

print(df_cust['Segment'].value_counts())

Mid-Value 344
Low-Value 157
High-Value 41
Name: Segment, dtype: int64